# Multi-level strain optimization approaches

Several strain design approaches use MILPs with nested optimization to enforce growth-coupled production. As the first nested optimization algorithm, OptKnock aimed to resolve the conflict between the microbial objective of fast growth with the engineering goal of fast production. It therefore constructs a max-max problem for the maximization of product synthesis under the assumption that the cell itself, will maximize its growth rate. One problem is that this formulation leads to overly optimistic strain designs, since it assumes that a cell would maximize production when attaining its maximal growth rate. In the worst case, however (potentially growth coupled prodcution), a cell might turn off production completely.

Successors of OptKnock, such as RobustKnock and OptCouple have overcome this problem since they guarantee production at maximum growth (weakly growth-coupled production). In the following sections we will explain how one can compute strain designs with OptKnock, RobustKnock and OptCouple, and how these methods can be combined with the minimial cut set approach.

In [ ]:
import straindesign as sd
import cobra

ecc = cobra.io.load_model('e_coli_core')

## OptKnock

Optknock is based on a bi-level optimization problem:

\begin{array}{ll}
    \text{maximize} \; & v_{production} \\
    \text{subject to}  & \begin{array}{ll}
        \text{maximize} &  v_{biomass} \hspace{10em} \\
        \text{subject to} & \mathbf{S~v=0} \\
        & v_{BM} \ge v_{BM}^{min} \\
        & (1-z_i)\cdot lb_i \le v_i \le (1-z_i)\cdot ub_i, ~\forall i\in\{1,\dots,n\}\\
        &\sum z_i \le \text{maxKOs}\\
        &z_i \in \{0,1\}
\end{array}\end{array}

The nested optimization is translated into a single-layer problem, so that it can be solved as a mixed-integer linear problem (MILP).

Translating the nested optimization into a single level optimization yields:

\begin{gathered}
\text{maximize} \; v_{production} \\
\text{subject to} \\
\begin{array}{lll}
    \begin{bmatrix}
        ~~\mathbf G & \mathbf 0 & \mathbf 0 \\
        ~~\mathbf D & \mathbf 0 & \mathbf 0 \\
        \mathbf{-c^\intercal}~ & \mathbf{g^\intercal} & \mathbf 0 \\
        ~~\mathbf 0 & \mathbf{G^\intercal} & \mathbf{I_{KO}} \\
    \end{bmatrix} &
    \begin{bmatrix}
        \mathbf{v} \\ \mathbf{y} \\ \mathbf{s}
    \end{bmatrix} &
    \begin{matrix}
        \le \\ \le \\ \le \\ =
    \end{matrix}
    \begin{bmatrix}\mathbf g \\ \mathbf d \\ 0 \\ \mathbf c\end{bmatrix} \\
\end{array} \\
\forall i: z_i = 1 \rightarrow v_i = 0 \\
\forall i: z_i = 0 \rightarrow s_i = 0 \\
\sum z_i \le MaxNoKO \\
\mathbf{y\ge 0}, \hskip 1em z\in\{0,1\}
\end{gathered}

## RobustKnock

## OptCouple

## Combined nested-MCS combinations